In [ ]:
# Setup
%pip install databricks-sdk==0.40.0 --quiet
dbutils.library.restartPython()

In [ ]:
# Initialize
%run ../_resources/00-setup $reset_all_data=false

import sys
sys.path.append('../_resources')
from gamification_framework import (
    init_learner,
    display_challenge_intro,
    display_challenge_success
)

learner = init_learner()
display_challenge_intro(
    challenge_name="Debugging Challenge: Fix the Broken Agents",
    difficulty="Intermediate → Advanced",
    points=150,
    description="Hunt down and fix 5 critical bugs in production agent code. Each bug represents real-world issues you'll encounter."
)

---

## 🐛 Bug #1: The Wrong ID Mystery

**Symptom**: Agent returns turbine IDs that don't exist in our database.

**Severity**: 🔴 Critical - Causes wrong turbines to be serviced!

**Your Task**: Find and fix the bug in the code below.

In [ ]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

# ❌ BUGGY CODE - Can you spot the problem?

turbine_lookup_prompt = """
You help technicians find turbine information.

When a user mentions a turbine, extract the ID.
Turbine IDs follow this format: WT-XXX where XXX is a 3-digit number.

If you can't find an exact match, suggest the closest match.
"""

def get_turbine_id(user_query: str) -> str:
    """Extract turbine ID from user query"""
    response = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[
            {"role": "system", "content": turbine_lookup_prompt},
            {"role": "user", "content": f"Extract turbine ID: {user_query}"}
        ],
        max_tokens=50
    )
    return response.choices[0].message.content

# Test cases
test_queries = [
    "Check turbine 42",
    "What's the status of the turbine at position 3?",
    "WT-999 is making noise"
]

print("🧪 Testing turbine ID extraction:\n")
for query in test_queries:
    result = get_turbine_id(query)
    print(f"Query: {query}")
    print(f"Result: {result}")
    print("-" * 60)

print("\n❓ What's wrong with this code?")
print("Hint: The agent is being TOO helpful...")

### 💡 Debugging Tips for Bug #1

Ask yourself:
1. What happens when the agent can't find an exact ID?
2. Is "suggesting the closest match" a good idea?
3. How should the agent handle ambiguous inputs?

### ✅ Your Fix:

In [ ]:
# 💪 YOUR TURN - Fix Bug #1

# TODO: Rewrite the prompt to be more strict
# Requirements:
# - Only return exact format matches (WT-XXX)
# - If no exact match, return "ERROR: No valid turbine ID found"
# - Never guess or suggest IDs
# - Validate that XXX is a number

fixed_turbine_lookup_prompt = """
# TODO: Your fixed prompt here
"""

def get_turbine_id_fixed(user_query: str) -> str:
    """Fixed version - extract turbine ID safely"""
    # TODO: Implement your fix
    pass

# Test your fix
print("🧪 Testing FIXED version:\n")
for query in test_queries:
    result = get_turbine_id_fixed(query)
    print(f"Query: {query}")
    print(f"Result: {result}")
    print("-" * 60)

---

## 🐛 Bug #2: The Crashing Cost Calculator

**Symptom**: Agent crashes with "AttributeError: 'NoneType' object has no attribute 'choices'"

**Severity**: 🟠 High - Prevents cost estimation functionality

**Your Task**: Find why the tool call is failing.

In [ ]:
# ❌ BUGGY CODE - This crashes unpredictably

# First, create a buggy UC function
spark.sql(f"""
CREATE OR REPLACE FUNCTION {catalog}.{schema}.calculate_repair_cost_buggy(
    parts_cost DOUBLE,
    labor_hours DOUBLE
)
RETURNS DOUBLE
COMMENT 'Calculate total repair cost'
RETURN parts_cost + (labor_hours * 85.0)
""")

cost_agent_prompt = """
You calculate repair costs using the calculate_repair_cost_buggy function.

When a user asks for cost estimation:
1. Extract parts cost and labor hours
2. Call the function
3. Return the result
"""

def estimate_cost_buggy(user_request: str):
    """Estimate repair cost - but crashes!"""
    try:
        response = w.serving_endpoints.query(
            name="databricks-meta-llama-3-1-70b-instruct",
            messages=[
                {"role": "system", "content": cost_agent_prompt},
                {"role": "user", "content": user_request}
            ],
            max_tokens=200
        )
        # Bug is here - assuming response is always valid
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Test it
test_requests = [
    "Cost for blade replacement: $25000 parts, 8 hours labor",
    "How much for sensor repair?",  # Missing data - will crash!
    "Cost estimate for gearbox"
]

print("🧪 Testing buggy cost calculator:\n")
for request in test_requests:
    result = estimate_cost_buggy(request)
    print(f"Request: {request}")
    print(f"Result: {result}")
    print("-" * 60)

print("\n❓ What's causing the crashes?")
print("Hint: The problem is missing error handling AND unclear instructions...")

In [ ]:
# 💪 YOUR TURN - Fix Bug #2

# TODO: Fix the issues:
# 1. Add proper error handling
# 2. Improve the prompt to handle missing data
# 3. Validate inputs before calling the function
# 4. Return helpful error messages

fixed_cost_agent_prompt = """
# TODO: Your improved prompt
# Should handle cases where user doesn't provide all data
"""

def estimate_cost_fixed(user_request: str):
    """Fixed cost estimation with proper error handling"""
    # TODO: Implement robust version
    pass

# Test your fix
print("🧪 Testing FIXED cost calculator:\n")
for request in test_requests:
    result = estimate_cost_fixed(request)
    print(f"Request: {request}")
    print(f"Result: {result}")
    print("-" * 60)

---

## 🐛 Bug #3: The Slow Response Problem

**Symptom**: Agent takes 30+ seconds to respond to simple queries

**Severity**: 🟡 Medium - Terrible user experience

**Your Task**: Profile and optimize the agent's performance.

In [ ]:
# ❌ SLOW CODE - Can you make it faster?

import time

def slow_status_check(turbine_id: str) -> dict:
    """Check turbine status - but VERY slow!"""
    start_time = time.time()
    
    # Step 1: Get sensor data (could be optimized)
    print("📊 Querying sensor data...")
    sensor_prompt = "Analyze sensor readings for a turbine and return structured data."
    sensor_response = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[{"role": "system", "content": sensor_prompt},
                  {"role": "user", "content": f"Get data for {turbine_id}"}],
        max_tokens=500  # Way too many!
    )
    sensor_data = sensor_response.choices[0].message.content
    
    # Step 2: Analyze the data (sequential - could be parallel?)
    print("🔍 Analyzing...")
    analysis_response = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[{"role": "system", "content": "Analyze this sensor data."},
                  {"role": "user", "content": sensor_data}],
        max_tokens=500
    )
    analysis = analysis_response.choices[0].message.content
    
    # Step 3: Generate recommendations (also sequential)
    print("💡 Generating recommendations...")
    rec_response = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[{"role": "system", "content": "Generate maintenance recommendations."},
                  {"role": "user", "content": analysis}],
        max_tokens=500
    )
    recommendations = rec_response.choices[0].message.content
    
    elapsed = time.time() - start_time
    
    return {
        "turbine_id": turbine_id,
        "sensor_data": sensor_data,
        "analysis": analysis,
        "recommendations": recommendations,
        "execution_time": f"{elapsed:.2f}s"
    }

# Test it
print("⏱️ Testing SLOW version:\n")
result = slow_status_check("WT-042")
print(f"\n⏱️ Total time: {result['execution_time']}")
print("\n❓ How can we make this faster?")
print("Hints: Look at max_tokens, sequential calls, and prompt complexity...")

In [ ]:
# 💪 YOUR TURN - Fix Bug #3

# TODO: Optimize for speed:
# 1. Reduce max_tokens to only what's needed
# 2. Combine prompts where possible
# 3. Use parallel execution for independent tasks
# 4. Cache repeated queries
# 5. Use more concise prompts

def fast_status_check(turbine_id: str) -> dict:
    """Optimized turbine status check"""
    start_time = time.time()
    
    # TODO: Implement optimized version
    # Target: < 10 seconds total
    
    elapsed = time.time() - start_time
    
    return {
        "turbine_id": turbine_id,
        "execution_time": f"{elapsed:.2f}s",
        "speedup": "TODO"
    }

# Test your optimized version
print("⚡ Testing OPTIMIZED version:\n")
result_fast = fast_status_check("WT-042")
print(f"\n⚡ Optimized time: {result_fast['execution_time']}")

---

## 🐛 Bug #4: The Security Leak

**Symptom**: Sensitive maintenance records appearing in logs

**Severity**: 🔴 Critical - GDPR/Security violation!

**Your Task**: Find and fix the data leakage.

In [ ]:
# ❌ INSECURE CODE - Leaking sensitive data!

# Sample sensitive data
maintenance_records = {
    "WT-042": {
        "technician": "John Smith",
        "phone": "+1-555-0123",
        "salary": "$85,000",
        "notes": "Employee ID: 12345, Access Code: ABC123"
    }
}

security_issue_prompt = """
You provide maintenance information to users.
When asked about a turbine, return ALL available information.
"""

def get_maintenance_info_insecure(turbine_id: str):
    """Get maintenance info - but exposes sensitive data!"""
    record = maintenance_records.get(turbine_id, {})
    
    # Bug: Sending ALL data to LLM (and logs!)
    full_context = f"""
    Turbine: {turbine_id}
    Technician: {record.get('technician')}
    Phone: {record.get('phone')}
    Salary: {record.get('salary')}
    Notes: {record.get('notes')}
    """
    
    response = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[
            {"role": "system", "content": security_issue_prompt},
            {"role": "user", "content": full_context}
        ],
        max_tokens=200
    )
    
    # Bug: Logging sensitive data
    print(f"DEBUG: Full record = {record}")
    
    return response.choices[0].message.content

# Test it
print("🔓 Testing INSECURE version:\n")
result = get_maintenance_info_insecure("WT-042")
print(result)

print("\n❓ What security issues do you see?")
print("Hints: PII in logs, unnecessary data exposure, no access control...")

In [ ]:
# 💪 YOUR TURN - Fix Bug #4

# TODO: Implement security best practices:
# 1. Filter sensitive fields before sending to LLM
# 2. Never log PII (names, phone numbers, IDs, etc.)
# 3. Add access control checks
# 4. Sanitize outputs
# 5. Add audit logging (without sensitive data)

def get_maintenance_info_secure(turbine_id: str, user_role: str = "technician"):
    """Secure version with proper data protection"""
    # TODO: Implement secure version
    pass

# Test your secure version
print("🔒 Testing SECURE version:\n")
result_secure = get_maintenance_info_secure("WT-042", user_role="technician")
print(result_secure)

---

## 🐛 Bug #5: The Hallucination Problem

**Symptom**: Agent invents maintenance procedures that don't exist

**Severity**: 🔴 Critical - Could cause equipment damage!

**Your Task**: Make the agent stick to verified information only.

In [ ]:
# ❌ HALLUCINATING CODE - Making up procedures!

hallucination_prompt = """
You are a maintenance expert. Help users fix turbine issues.
Provide detailed step-by-step instructions.
"""

def get_procedure_buggy(issue: str):
    """Get maintenance procedure - but hallucinates!"""
    response = w.serving_endpoints.query(
        name="databricks-meta-llama-3-1-70b-instruct",
        messages=[
            {"role": "system", "content": hallucination_prompt},
            {"role": "user", "content": f"How do I fix: {issue}"}
        ],
        max_tokens=400
    )
    return response.choices[0].message.content

# Test it
test_issues = [
    "Gearbox overheating",
    "Quantum fluctuations in the flux capacitor",  # Nonsense! Will it detect?
    "Blade resonance at 42 Hz"
]

print("🤖 Testing HALLUCINATING version:\n")
for issue in test_issues:
    result = get_procedure_buggy(issue)
    print(f"Issue: {issue}")
    print(f"Response: {result}")
    print("-" * 60)

print("\n❓ How can we prevent hallucinations?")
print("Hints: RAG, knowledge boundaries, confidence scoring...")

In [ ]:
# 💪 YOUR TURN - Fix Bug #5

# TODO: Implement anti-hallucination measures:
# 1. Integrate RAG (retrieve from verified docs)
# 2. Add knowledge boundary detection
# 3. Require citations/sources
# 4. Add confidence scoring
# 5. Escalate unknown issues to humans

grounded_prompt = """
# TODO: Create prompt that:
# - Only answers based on provided context
# - Says "I don't know" when information is unavailable
# - Cites sources for all procedures
# - Flags non-standard issues for expert review
"""

def get_procedure_grounded(issue: str, knowledge_base: dict):
    """Grounded procedure lookup - no hallucinations!"""
    # TODO: Implement RAG-based version
    pass

# Test your grounded version
verified_procedures = {
    "gearbox overheating": "Official procedure doc: PROC-GB-001",
    "blade resonance": "Official procedure doc: PROC-BL-015"
}

print("✅ Testing GROUNDED version:\n")
for issue in test_issues:
    result = get_procedure_grounded(issue, verified_procedures)
    print(f"Issue: {issue}")
    print(f"Response: {result}")
    print("-" * 60)

---

## 📊 Final Evaluation

Time to check if you fixed all the bugs!

In [ ]:
# Automated bug verification

def verify_all_fixes():
    """Check if all bugs are fixed"""
    bugs_fixed = 0
    total_bugs = 5
    
    print("🔍 Verifying bug fixes...\n")
    
    # Bug #1: Check if it handles invalid IDs properly
    try:
        result = get_turbine_id_fixed("Check turbine 42")
        if "ERROR" in result or "WT-" not in result:
            print("✅ Bug #1 FIXED: Rejects invalid IDs")
            bugs_fixed += 1
        else:
            print("❌ Bug #1 NOT FIXED: Still accepting invalid IDs")
    except:
        print("❌ Bug #1 NOT FIXED: Function not implemented")
    
    # Bug #2: Check error handling
    try:
        result = estimate_cost_fixed("Cost for something")
        if result and "error" not in result.lower():
            print("✅ Bug #2 FIXED: Handles missing data gracefully")
            bugs_fixed += 1
        else:
            print("❌ Bug #2 NOT FIXED: Still crashes on edge cases")
    except:
        print("❌ Bug #2 NOT FIXED: Function not implemented")
    
    # Bug #3: Check performance
    try:
        import time
        start = time.time()
        result = fast_status_check("WT-042")
        elapsed = time.time() - start
        if elapsed < 15:
            print(f"✅ Bug #3 FIXED: Optimized to {elapsed:.2f}s")
            bugs_fixed += 1
        else:
            print(f"❌ Bug #3 NOT FIXED: Still slow ({elapsed:.2f}s)")
    except:
        print("❌ Bug #3 NOT FIXED: Function not implemented")
    
    # Bug #4 & #5: Manual review required
    print("\n⚠️  Bugs #4 and #5 require manual code review")
    print("    Check your implementations above for:")
    print("    - No PII in logs (Bug #4)")
    print("    - RAG integration (Bug #5)")
    
    print(f"\n📊 Bugs Fixed: {bugs_fixed}/{total_bugs} automated checks")
    
    return bugs_fixed

bugs_fixed = verify_all_fixes()

if bugs_fixed >= 3:
    print("\n🎉 Great debugging work!")
    learner.complete_challenge("debugging_challenge", points=150)
    learner.award_badge("debugging_hero")
    display_challenge_success("Debugging Challenge", 150)
else:
    print("\n💡 Keep working on the fixes!")

---

## 🎓 Key Takeaways

You've learned to debug:

✅ **Over-Helpful Agents** - Being too flexible causes errors  
✅ **Error Handling** - Always validate inputs and outputs  
✅ **Performance Issues** - Optimize tokens, parallelize, cache  
✅ **Security Leaks** - Never log PII, filter sensitive data  
✅ **Hallucinations** - Use RAG, set boundaries, require sources

### 🏆 Debugging Best Practices

1. **Start with small test cases** - Isolate the bug
2. **Add logging strategically** - But never log PII!
3. **Validate all inputs** - Never trust user data
4. **Profile before optimizing** - Measure first
5. **Use RAG for facts** - Don't let agents guess

---

## 🚀 Next Steps

Apply your debugging skills:

- **05.X-real-world-scenarios**: Debug production emergencies
- **05.Y-performance-optimization**: Deep dive into agent performance
- **Your own projects**: You're now ready to debug real agents!

In [ ]:
# Check your overall progress
learner.display_progress()